In [1]:
import numpy as np
import pandas as pd
import csv
import nltk

In [2]:
import os
import operator
import gensim
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from os import listdir
from os.path import isfile, join
from os import walk

In [3]:
import re

def urlify(s):

    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)

    # Replace all runs of whitespace with a single dash
    # s = re.sub(r"\s+", '-', s)

    return s

# Prints: I-cant-get-no-satisfaction"
print(urlify("I can't get no satisfaction!"))

I cant get no satisfaction


In [6]:
data = pd.read_csv("masterfile.csv")

In [7]:
fuzzy_data = data.copy()

In [8]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

In [9]:
print(os.getcwd())
path = "/home/akhil/Downloads/Kubric_winter_internship"
os.chdir(path)
print(os.getcwd())

/home/akhil/Downloads/Kubric_winter_internship
/home/akhil/Downloads/Kubric_winter_internship


In [10]:
row_count = len(data.index)
col_count = len(data.columns)

print(row_count)
print(col_count)

261
72


In [10]:
droplist_one = [i for i in data.columns if data[i].dtype == float or data[i].dtype == int]
print("The list of columns with int or float value which needs to be dropped ",droplist_one)
data.drop(droplist_one,axis=1,inplace=True)
print("data.shape is ",data.shape)
row_count = len(data.index)
col_count = len(data.columns)

print("Row count is ",row_count)
print("Column count is ",col_count)

The list of columns with int or float value which needs to be dropped  ['Unnamed: 58', 'segmentid_fb', 'Coming soon Budget ']
data.shape is  (261, 69)
Row count is  261
Column count is  69


In [33]:
print("row_count is", row_count)
print("col_count is", col_count)

imagelist = []
videolist = []

for j in range(0,col_count):
    garbage = []
    garbage= [i for i in data.iloc[:,j] if ".jpg" in i or ".jpeg" in i or ".png" in i or ".PNG" in i or ".JPEG" in i or ".Jppeg" in i]
    imagelist.extend(garbage)
#print("The list of images with image input\n",imagelist)
ccc = len(imagelist)/row_count
print("No of image items in the sheet are",len(imagelist))
print("Number of columns with image input is ",ccc)

for j in range(0,col_count):
    garbage = []
    garbage= [i for i in data.iloc[:,j] if ".mov" in i or ".mp4" in i or ".png" in i or ".PNG" in i or ".JPEG" in i or ".Jpeg" in i]
    videolist.extend(garbage)


No of image items in the sheet are 3761
row_count is 261
col_count is 69
Number of columns with image input is  14.409961685823754


In [53]:
#image
image_fuzzy_words = ["image","Image","img","logo","Logo","Img"]
imagelist = [i for i in fuzzy_data.columns if any(ext in i for ext in image_fuzzy_words)]
fuzzy_image = fuzzy_data.loc[:,imagelist]
print(imagelist)

['Frame1_Image1', 'Frame2_image', 'Frame3_category 1 - image', 'Frame3_category 1 - image_brand1', 'Frame3_category 1 - image_brand2', 'Frame3_category 2 - image', 'Frame3_category 2 - image_brand1', 'Frame3_category 2 - image_brand2', 'Frame3_category 3 - image', 'Frame3_category 3 - image_brand1', 'Frame3_category 3 - image_brand2', 'Frame3_EORSLogo', 'Frame4_HeroProduct1-image', 'Frame4_HeroProduct2-image', 'Frame4_HeroProduct3-image', 'Frame4_EORSLogo', 'Frame5_image1_banklogo', 'Frame5_EORSLogo']


In [67]:
#image
image_fuzzy_words = ["image","Image","img","logo","Logo","Img"]
imagelist = [i for i in fuzzy_data.columns if any(ext in i for ext in image_fuzzy_words)]
fuzzy_image = fuzzy_data.loc[:,imagelist]

#video
video_fuzzy_words = ["video","effect", "vo","box_call_out", "VO"]
videolist = [i for i in fuzzy_data.columns if any(ext in i for ext in image_fuzzy_words)]
fuzzy_video = fuzzy_data.loc[:,videolist]

#audio
audiolist = []
audio_fuzzy_words = ["audio","music","bg","Audio","AUDIO"]
audiolist = [i for i in data.columns if any(ext in i for ext in audio_fuzzy_words) and "video" not in i]
fuzzy_audio = fuzzy_data.loc[:,audiolist]

#text
textlist = []
text_fuzzy_words = ["text","effect","product_name", "price" ,"location", "Location", "Campaign", "gender","name"]
textlist = [i for i in fuzzy_data.columns if any(ext in i for ext in text_fuzzy_words)]
fuzzy_text = fuzzy_data.loc[:,textlist]

#int and float columns
droplist_one = [i for i in fuzzy_data.columns if fuzzy_data[i].dtype == float or fuzzy_data[i].dtype == int]

#others
#to create others bucket we will simply delete the cols which have been used above
fuzzy_random = fuzzy_data.copy()
droplist = imagelist + videolist + audiolist + textlist + droplist_one
fuzzy_random.drop(droplist,axis=1,inplace=True)

#Now adding the directory columns to all the sub-dataframes

data = data.reset_index()
fuzzy_image = fuzzy_image.reset_index()
fuzzy_video = fuzzy_video.reset_index()
fuzzy_audio = fuzzy_audio.reset_index()
fuzzy_text = fuzzy_text.reset_index()
fuzzy_random = fuzzy_random.reset_index()

data.index = fuzzy_image.index

#Adding the directory column to all the sub-dataframes
fuzzy_image['directory'] = data['directory']
fuzzy_video['directory'] = data['directory']
fuzzy_audio['directory'] = data['directory']
fuzzy_text['directory'] = data['directory']

In [85]:
#Now we have a subset with the columns which contain the images and the directory list in which the image
#must be present

garbage_variable = os.getcwd()
print(garbage_variable)

for i in range(0,row_count):
    mypath = garbage_variable + "/" + fuzzy_image[["directory"]][i]
    garbage_list = []
    for (dirpath, dirnames, filenames) in walk(mypath):
        garbage_list.extend(filenames)
        break
    
print(mypath)

In [55]:
#image
image_fuzzy_words = ["image","Image","img","logo","Logo","Img"]
imagelist = [i for i in fuzzy_data.columns if any(ext in i for ext in image_fuzzy_words)]
fuzzy_image = fuzzy_data.loc[:,imagelist]

print("The list of cols with image input\n",imagelist)
print("No of image items in the sheet are",len(imagelist))
print("\n")
print("fuzzy_image shape is",fuzzy_image.shape)
print("fuzzy_data shape is",fuzzy_data.shape)
print("\n")

#video
video_fuzzy_words = ["video","effect", "vo","box_call_out", "VO"]
videolist = [i for i in fuzzy_data.columns if any(ext in i for ext in image_fuzzy_words)]
fuzzy_video = fuzzy_data.loc[:,videolist]

print("The list of cols with video input\n",videolist)
print("No of video items in the sheet are",len(videolist))
print("\n")
print("fuzzy_video shape is",fuzzy_video.shape)
print("fuzzy_data shape is",fuzzy_data.shape)
print("\n")

#audio
audiolist = []
audio_fuzzy_words = ["audio","music","bg","Audio","AUDIO"]
audiolist = [i for i in data.columns if any(ext in i for ext in audio_fuzzy_words) and "video" not in i]
fuzzy_audio = fuzzy_data.loc[:,audiolist]

print("The list of cols with audio input\n",audiolist)
print("No of audio items in the sheet are",len(audiolist))
print("\n")
print("fuzzy_audio shape is",fuzzy_audio.shape)
print("fuzzy_data shape is",fuzzy_data.shape)
print("\n")


#text
textlist = [i for i in fuzzy_data.columns if "text" in i and "effect" not in i or "product_name" in i or "price" in i and fuzzy_data[i].dtype == object or "location" in i or "Location" in i or "Campaign" in i or "gender" in i or "name" in i]
fuzzy_text = fuzzy_data.loc[:,textlist]

print("The list of cols with text input\n",textlist)
print("No of text items in the sheet are",len(textlist))
print("\n")
print("fuzzy_text shape is",fuzzy_text.shape)
print("fuzzy_data shape is",fuzzy_data.shape)
print("\n")

#int and float columns
droplist_one = [i for i in fuzzy_data.columns if fuzzy_data[i].dtype == float or fuzzy_data[i].dtype == int]

print("The list of cols with numbers i.e the droplist_one is \n",droplist_one)
print("No of droplist items in the sheet are",len(droplist_one))
print("\n")

#others
#to create others bucket we will simply delete the cols which have been used above
fuzzy_random = fuzzy_data.copy()
droplist = imagelist + videolist + audiolist + textlist + droplist_one
fuzzy_random.drop(droplist,axis=1,inplace=True)

print("droplist\n", droplist)
print("the length of droplist is ",len(droplist))
print("the shape of fuzzy random is", fuzzy_random.shape)
print("\n")

The list of cols with image input
 ['Frame1_Image1', 'Frame2_image', 'Frame3_category 1 - image', 'Frame3_category 1 - image_brand1', 'Frame3_category 1 - image_brand2', 'Frame3_category 2 - image', 'Frame3_category 2 - image_brand1', 'Frame3_category 2 - image_brand2', 'Frame3_category 3 - image', 'Frame3_category 3 - image_brand1', 'Frame3_category 3 - image_brand2', 'Frame3_EORSLogo', 'Frame4_HeroProduct1-image', 'Frame4_HeroProduct2-image', 'Frame4_HeroProduct3-image', 'Frame4_EORSLogo', 'Frame5_image1_banklogo', 'Frame5_EORSLogo']
No of image items in the sheet are 18


fuzzy_image shape is (261, 18)
fuzzy_data shape is (261, 72)


The list of cols with video input
 ['Frame1_Image1', 'Frame2_image', 'Frame3_category 1 - image', 'Frame3_category 1 - image_brand1', 'Frame3_category 1 - image_brand2', 'Frame3_category 2 - image', 'Frame3_category 2 - image_brand1', 'Frame3_category 2 - image_brand2', 'Frame3_category 3 - image', 'Frame3_category 3 - image_brand1', 'Frame3_category 3 

In [113]:
mypath = "/home/akhil/Downloads/"
f = []
for (dirpath, dirnames, filenames) in walk(mypath):
    f.extend(filenames)
    break
    
print("the files are \n",f)
print("the length of f is ",len(f))

the files are 
 ['alpha_zero.pdf', '(1) Annie Duke-Thinking in bets _ making smarter decisions when you don’t have all the facts.epub', 'Deficit.ppt', 'HEC_Business_Game_2019.vcf', 'J.K. Rowling-Fantastic Beasts and Where to Find Them-Bloomsbury (2001).mobi', 'Postman-linux-x64-6.1.3 (1).tar.gz', 'generated-bedrooms.png', 'cuda_9.0.176_384.81_linux.run', '1.UTB - F - C1 - UTB - F - C1-Final.csv', 'Timothy Falcon Crack-Heard on the Street_ Quantitative Questions from Wall Street Job Interviews (2003).pdf', 'multithreading_test.py', 'config.ini', 'akhil.psd', 'J.K. Rowling-Fantastic Beasts and Where to Find Them-Bloomsbury (2001).pdf', 'Postman-linux-x64-6.1.3.tar.gz', 'Michael Lewis-The Undoing Project_ A Friendship That Changed Our Minds-W. W. Norton & Company (2016).epub', 'linux-firmware-nonfree_1.14ubuntu1_all.deb', 'akhil.jpg', '48386645_366038487274509_2283414986346725376_n.png', 'chess-m.jpg', 'generator.py', '_MG_6096 (2)-min.JPG', 'Michael Lewis-The Undoing Project_ A Friendshi

In [77]:
#fuzzy_image
raw_image_documents = []
for i in list(range(1,len(imagelist))):
    raw_image_documents.extend(fuzzy_image.iloc[:,i])
print("length of image vector is ",len(raw_image_documents))
print("the length of image list is ", len(imagelist))

raw_image_documents[0]

length of image vector is  4437
the length of image list is  18


'image'

In [102]:
mypath = "/home/akhil/Downloads/Kubric_winter_internship"
list_aa = []
for (dirpath, dirnames, filenames) in walk(mypath):
    list_aa.extend(filenames)
    break
    
print("list under assets folder one ", list_aa)

list under assets folder one  ['FB Personalized Ads_Images-20181217T081726Z-001.zip', 'Speech to text using google speech API.ipynb', 'code a.ipynb', 'startup-3.jpg', 'sphinx4-5prealpha-src.zip', 'Different_codes.ipynb', 'Accuracy, Precision, Recall or F1_ – Towards Data Science.pdf', 'FB Personalized Ads_Images-20181221T075900Z-001.zip']


In [76]:
list_aa = []
aaa = 0
bbb= 0
for directories in os.listdir("/home/akhil/Downloads/Kubric_winter_internship/assets_folder_one/"): 
    mypath = "/home/akhil/Downloads/Kubric_winter_internship/assets_folder_one/" + directories + "/"
    for (dirpath, dirnames, filenames) in walk(mypath):
        list_aa.extend(filenames)
        aaa = aaa+1
        break
for directories in os.listdir("/home/akhil/Downloads/Kubric_winter_internship/assets_folder_two/"): 
    mypath2 = "/home/akhil/Downloads/Kubric_winter_internship/assets_folder_two/" + directories + "/"
    for (dirpath, dirnames, filenames) in walk(mypath2):
        list_aa.extend(filenames)
        bbb = bbb +1
        break
               
print("The number of folders in the assets_folder_one are ",aaa)
print("The number of folders in the assets_folder_two are ",bbb)
print("\n")    
# print(list_aa)
# print("\n") 
print("the length of list_aa is", len(list_aa))
#     dir = os.path.join('/home/akhil/Downloads/Kubric_winter_internship/assets_folder_one', directories)
#     os.chdir(dir)
#     current = os.path.dirname(dir)
#     new = str(current).split("-")[0]
#     print(new)

The number of folders in the assets_folder_one are  30
The number of folders in the assets_folder_two are  12


the length of list_aa is 314


In [74]:
list_aa=local_list.copy()
raw_image_documents =csv_images.copy()

NameError: name 'local_list' is not defined

In [78]:
#list_aa is the list of names of all files in the remote computer
#raw_image_documents is the list of all images in the csv file

#This cell will convert all the _ in the above strings to space so that the tokens can work properly
#firstly creating duplicates of those string lists so that we can check for any mistakes 

local_list = list_aa.copy()
csv_images= raw_image_documents.copy()
aa=0

for i in range(1,len(list_aa)):
    list_aa[i]=list_aa[i].replace("_"," ")
    list_aa[i]=list_aa[i].replace("."," ")
    
for i in range(1,len(raw_image_documents)):
    raw_image_documents[i]=raw_image_documents[i].replace("_"," ")
    raw_image_documents[i]=raw_image_documents[i].replace("."," ")


    
print(local_list[2])

AttributeError: 'float' object has no attribute 'replace'

In [84]:
raw_image_documents
raw_image_documents = [x for x in raw_image_documents if str(x) != 'nan']
list_aa
# raw_image_documents = [x for x in raw_image_documents if str(x) != 'nan']


['WUS_Date.jpg',
 'WUS Perfume jpg',
 'WUS Jeans jpg',
 'UTB Watch M jpg',
 'WUS CasualShoes jpg',
 'WUS jpg',
 'WUS Shirt jpg',
 'WUS Jacket jpg',
 'UVS Makeup jpg',
 'UVS FTop F jpg',
 'UVS F jpg',
 'UVS Festival F jpg',
 'UVS FJacket F jpg',
 'UVS FKurta F jpg',
 'UVS Heels F jpg',
 'UVS Sweater F jpg',
 'UVS FDress F jpg',
 'WUS Tshirt jpg',
 'WUS Jeans jpg',
 'WUS CasualShoes jpg',
 'WUS jpg',
 'WUS Work(1) jpg',
 'WUS Jacket jpg',
 'UTB Tops F(1) jpg',
 'UTB Kurta F jpg',
 'UTB Dresses F jpg',
 'UTB F jpg',
 'UTB Trousers F jpg',
 'UTB Tops F jpg',
 'UTB Flats F jpg',
 'UVS Watch M jpg',
 'UVS Shirt M jpg',
 'UVS Wedding M jpg',
 'UVS Jeans M jpg',
 'UVS WShirt M jpg',
 'UVS WJacket jpg',
 'TSG Vacation jpg',
 'TSG Kurta jpg',
 'TSG VTop jpg',
 'TSG Flats jpg',
 'TSG jpg',
 'TSG SweatShirt jpg',
 'TSG VDress jpg',
 'TSG Makeup jpg',
 'SP Tshirt jpg jpg',
 'SP CasualShoes jpg',
 'SP SportsShoes jpg',
 'SP Jeans jpg',
 'SP Shirt jpg jpg',
 'SP Trousers jpg',
 'SP Family jpg',
 'SP 

In [88]:
solutions = []
for i in range(0,len(raw_image_documents)):
    #print("The input query is",raw_image_documents[i])
    solutions.append(process.extractOne(raw_image_documents[i], list_aa, scorer=fuzz.token_sort_ratio))
print(solutions)


[('FHM KBag jpg', 35), ('TSG Vacation jpg', 47), ('UTB Brunch M jpg', 100), ('UTB Brunch M jpg', 100), ('UTB Brunch M jpg', 100), ('UTB Workwear M jpg', 100), ('UTB Workwear M jpg', 88), ('UTB Workwear M jpg', 88), ('UTB Party jpg', 100), ('UTB Party jpg', 100), ('UTB Party jpg', 100), ('UTB Vacation M jpg', 100), ('UTB Vacation M jpg', 100), ('UTB Vacation M jpg', 100), ('UTB Brunch F jpg', 100), ('UTB Brunch F jpg', 100), ('UTB Brunch F jpg', 100), ('UTB Workwear F jpg', 100), ('UTB Workwear F jpg', 100), ('UTB Workwear F jpg', 100), ('UTB Party F jpg', 100), ('UTB Party F jpg', 100), ('UTB Party F jpg', 100), ('UTB Vacation F jpg', 100), ('UTB Vacation F jpg', 100), ('UTB Kurta F jpg', 100), ('WUS Work jpg', 100), ('WUS Work jpg', 100), ('WUS Work jpg', 100), ('WUS Date jpg', 100), ('WUS Date jpg', 100), ('WUS Date jpg', 100), ('WUS Hangout jpg', 100), ('WUS Hangout jpg', 100), ('WUS Hangout jpg', 100), ('WUS Weekend jpg', 100), ('WUS Weekend jpg', 100), ('WUS Weekend jpg', 100), ('

In [91]:
#raw_image_documents
print(len(raw_image_documents))
print(len(solutions))
list(zip(raw_image_documents , solutions))

# a=[]
# for i in list(range(1,len(f))):
#     a.append(fuzz.token_set_ratio(f[i],raw_image_documents[8]))
#     #print("the token set ratio is ",a)
    
# print(a)

4416
4416


[('image', ('FHM KBag jpg', 35)),
 ('occasion image', ('TSG Vacation jpg', 47)),
 ('UTB Brunch M jpg', ('UTB Brunch M jpg', 100)),
 ('UTB Brunch M jpg', ('UTB Brunch M jpg', 100)),
 ('UTB Brunch M jpg', ('UTB Brunch M jpg', 100)),
 ('UTB Workwear M jpg', ('UTB Workwear M jpg', 100)),
 ('UTB Workwear M', ('UTB Workwear M jpg', 88)),
 ('UTB Workwear M', ('UTB Workwear M jpg', 88)),
 ('UTB Party jpg', ('UTB Party jpg', 100)),
 ('UTB Party jpg', ('UTB Party jpg', 100)),
 ('UTB Party jpg', ('UTB Party jpg', 100)),
 ('UTB Vacation M jpg', ('UTB Vacation M jpg', 100)),
 ('UTB Vacation M jpg', ('UTB Vacation M jpg', 100)),
 ('UTB Vacation M jpg', ('UTB Vacation M jpg', 100)),
 ('UTB Brunch F jpg', ('UTB Brunch F jpg', 100)),
 ('UTB Brunch F jpg', ('UTB Brunch F jpg', 100)),
 ('UTB Brunch F jpg', ('UTB Brunch F jpg', 100)),
 ('UTB Workwear F jpg', ('UTB Workwear F jpg', 100)),
 ('UTB Workwear F jpg', ('UTB Workwear F jpg', 100)),
 ('UTB Workwear F jpg', ('UTB Workwear F jpg', 100)),
 ('UTB Part

In [ ]:
data['Item Code'].astype(str)

In [38]:
data.iloc[:,5].head()

0    UTB_Male_Brunch_Images-UTB_M.jpg
1    UTB_Male_Brunch_Images-UTB_M.jpg
2    UTB_Male_Brunch_Images-UTB_M.jpg
3         UTB_Male_Workwear-UTB_M.jpg
4         UTB_Male_Workwear-UTB_M.jpg
Name: s1/callout_image, dtype: object

In [69]:
sub_data.iloc[:,4].head(3)

0    Want to sport a dapper new look\nthe next time...
1    Want to sport a dapper new look\nthe next time...
2    Want to sport a dapper new look\nthe next time...
Name: s2/caption_text, dtype: object

In [76]:
for i in list(range(1,43)):
    sub_data.iloc[:,i] = sub_data.iloc[:,i].astype(str)

In [90]:
df = sub_data.apply(''.join)

In [12]:
raw_documents = []
for i in list(range(1,44)):
    raw_documents.extend(sub_data.iloc[:,i])

NameError: name 'sub_data' is not defined

In [11]:
# Below this cell the code is to get the score for similarity between two sentences using gensim library
# This is not very important as fuzzywuzzy provides a better approach

In [101]:
from nltk.tokenize import word_tokenize
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]
#print(gen_docs)

In [103]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary[5])
print("Number of words in dictionary:",len(dictionary))
for i in range(len(dictionary)):
    print(i, dictionary[i])

.
Number of words in dictionary: 356
0 ,
1 \ngood
2 hey
3 looking
4 !
5 .
6 \nmr
7 dapper
8 there
9 globetrotter
10 hello
11 utb_male_brunch_images-utb_m.jpg
12 utb_male_workwear-utb_m.jpg
13 utb_parties_male-utb_m.jpg
14 utb_vacation_male-utb_m.jpg
15 utb_male_brunch_images-utb_brunch_m.jpg
16 utb_male_workwear-utb_workwear_m.jpg
17 utb_parties_male-utb_party.jpg
18 utb_vacation_male-utb_vacation_m.jpg
19 ?
20 a
21 head
22 look\nthe
23 new
24 next
25 out
26 sport
27 time
28 to
29 want
30 you
31 \nworkwear
32 essentials
33 for
34 's
35 2019
36 it
37 party\nlike
38 ready
39 are
40 haves
41 here
42 must
43 some\nvacation
44 utb_male_brunch_images-utb_sportsshoes_m.jpg
45 utb_male_brunch_images-utb_watch_m.jpg
46 utb_male_brunch_images-utb_tshirt_m.jpg
47 utb_male_workwear-utb_workshirts_m.jpg
48 utb_male_workwear-utb_jacket_m.jpg
49 utb_parties_male-utb_watch_m.jpg
50 utb_parties_male-utb_shirt_m.jpg
51 utb_parties_male-utb_jacket_m.jpg
52 utb_vacation_male-utb_sportsshoes_m.jpg
53 utb_v

In [104]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (2, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(0, 1), (2, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(0, 1), (2, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(0, 1), (2, 1), (8, 1)], [(0, 1), (2, 1), (8, 1)], [(0, 1), (2, 1), (8, 1)], [(4, 1), (6, 1), (9, 1), (10, 1)], [(4, 1), (6, 1), (9, 1), (10, 1)], [(4, 1), (6, 1), (9, 1), (10, 1)], [(11, 1)], [(11, 1)], [(11, 1)], [(12, 1)], [(12, 1)], [(12, 1)], [(13, 1)], [(13, 1)], [(13, 1)], [(14, 1)], [(14, 1)], [(14, 1)], [(15, 1)], [(15, 1)], [(15, 1)], [(16, 1)], [(16, 1)], [(16, 1)], [(17, 1)], [(17, 1)], [(17, 1)], [(18, 1)], [(18, 1)], [(18, 1)], [(7, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)], [(7, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)], [(7, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (

In [105]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)

TfidfModel(num_docs=516, num_nnz=2705)
2705


In [106]:
query_doc = [w.lower() for w in word_tokenize("Socks are a force for good.")]
print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

['socks', 'are', 'a', 'force', 'for', 'good', '.']
[(5, 1), (20, 1), (33, 1), (39, 1), (151, 1)]
[(5, 0.33630764978306377), (20, 0.5376708376793825), (33, 0.3777894096298184), (39, 0.407421731486242), (151, 0.5376708376793825)]


In [107]:
sims[query_doc_tf_idf]

NameError: name 'sims' is not defined

In [ ]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print sims # print sorted (document number, similarity score) 2-tuples